In [4]:
%load_ext autoreload
%autoreload 2

import credentials
import os

In [8]:
creds = credentials.load_spotify_creds('../credentials.json')
os.environ["SPOTIPY_CLIENT_ID"] = creds['client_id']
os.environ["SPOTIPY_CLIENT_SECRET"] = creds['client_secret']

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

results = sp.search(q='weezer', limit=20)
for idx, track in enumerate(results['tracks']['items']):
    print(idx, track['name'])

0 Island In The Sun
1 Say It Ain't So
2 Buddy Holly
3 Beverly Hills
4 Lost in the Woods - Weezer Version
5 Undone - The Sweater Song
6 Africa
7 Hash Pipe
8 Hero
9 My Name Is Jonas
10 Feels Like Summer
11 Pork And Beans
12 Take on Me
13 El Scorcho
14 Perfect Situation
15 Everybody Wants to Rule the World
16 Only In Dreams
17 The End of the Game
18 Mr. Blue Sky
19 Pacific Coast Highway In The Movies (feat. Rivers Cuomo of Weezer)
